# Measuring membrane thicknesses in situ: a Jupyter notebook guide

In [1]:
%load_ext autoreload
%autoreload 2

## Load dependencies

If you have a configured cryoCAT environment, following the instructions in the README at [https://github.com/turonova/cryoCAT](https://github.com/turonova/cryoCAT), you will need to install the following packages in your environment for all imports to work corectly.

```bash
pip install pyyaml tqdm
conda install numba -c conda-forge
```

In [2]:
import pandas as pd
from cryocat import memthick

## Input data

Example instance segmentation of a *Chlamydomonas reinhardtii* tomogram [1], containing nuclear envelope (NE), inner and outer mitochondrial membranes (IMM and OMM), generated using [MemBrain-seg](https://github.com/teamtomo/membrain-seg/tree/main) [2] with enabled connected components, is available [here](https://oc.biophys.mpg.de/owncloud/apps/files/files/13857656?dir=/cryocat_inputs/membrane_thickness/inputs).

An example configuration file specifying the instance segmentation label and the membrane of interest it belongs to can be found in the `inputs` folder or [here](https://oc.biophys.mpg.de/owncloud/apps/files/files/13857656?dir=/cryocat_inputs/membrane_thickness/inputs).

The pipeline can be run on semantic segmentations as well (i.e., all membrane instances are labeled with ones).

#### References:

[1] S. Khavnekar, *et al.* Towards the Visual Proteomics of C. reinhardtii using High-throughput Collaborative in situ Cryo-ET. *Microscopy and Microanalysis*. **2023**. 29:961-963. [https://doi.org/10.1093/micmic/ozad067.480](https://doi.org/10.1093/micmic/ozad067.480)

[2] L. Lamm, *et al.* MemBrain v2: an end-to-end tool for the analysis of membranes in cryo-electron tomography. *bioRxiv*. **2024**. [https://doi.org/10.1101/2024.01.05.574336](https://doi.org/10.1101/2024.01.05.574336)


## Overview

This notebook demonstrates the membrane thickness analysis workflow using the `membrane_thickness.py` tool. The pipeline extracts surface points from membrane segmentations, assigns them to inner and outer leaflets, and measures the thickness between corresponding points.

## Parameters and Configuration

### Input Files

| Parameter | Description | Default |
|-----------|-------------|---------|
| `segmentation_path` | Input segmentation MRC file (with correct pixel size) | (required) |
| `output_dir` | Directory for output files | Same directory as input |
| `config_path` | YAML configuration file for multiple membrane labels | `None` |
| `membrane_labels` | Dictionary mapping names to label values (e.g., `{"NE": 1, "ER": 2}`) | `{"membrane": 1}` |

> **Note:** If neither `config_path` nor `membrane_labels` are provided, the tool assumes a semantic segmentation where all non-zero voxels belong to the membrane of interest.

### Surface Point Processing

| Parameter | Description | Default | Recommended |
|-----------|-------------|---------|------------|
| `interpolate` | Enable point interpolation for better surface coverage | `True` | ✓ |
| `interpolation_points` | Number of points to add between adjacent vertices | `1` | Adjust based on desired sampling |
| `refine_normals` | Apply weighted-average normal refinement | `True` | ✓ |
| `radius_hit` | Neighborhood radius for normal refinement (voxels) | `10` | Adjust based on resolution |
| `flip_normals` | Orient normals to point inward | `True` | ✓ Required for thickness measurement |

### Thickness Measurement

| Parameter | Description | Default | Recommended |
|-----------|-------------|---------|------------|
| `max_thickness` | Maximum search distance (nm) | `8.0` | Adjust based on expected thickness |
| `max_angle` | Maximum cone search angle (degrees) | `3.0` | Lower values (1-3°) give more accurate results |
| `direction` | Direction of measurement (`"1to2"` or `"2to1"`) | `"1to2"` | Try both for comparison |

### Hardware Options

| Parameter | Description | Default |
|-----------|-------------|---------|
| `use_gpu` | Use GPU acceleration if available | `True` |
| `use_cpu` | Force CPU implementation | `False` |
| `cpu_threads` | Number of CPU threads for parallel processing | All available |
| `batch_size` | Processing batch size | `2000` |

### Pipeline Control

| Parameter | Description | Default |
|-----------|-------------|---------|
| `mode` | Processing mode: `"full"`, `"surface"`, or `"thickness"` | `"full"` |
| `input_csv` | CSV file with surface points (required for `mode="thickness"`) | `None` |

### Output Options

| Parameter | Description | Default |
|-----------|-------------|---------|
| `save_vertices_mrc` | Save vertices as MRC for visualization | `False` |
| `save_vertices_xyz` | Save vertices as XYZ point cloud | `False` |
| `save_thickness_mrc` | Save thickness values as MRC volume | `False` |

## Output Files

All output files generated in this tutorial can be found [here](https://oc.biophys.mpg.de/owncloud/apps/files/files/13857682?dir=/cryocat_inputs/membrane_thickness/outputs)

For each membrane, the pipeline generates:

| File | Description |
|------|-------------|
| `*_vertices_normals.csv` | Surface points with coordinates, normals and surface assignments |
| `*_thickness.csv` | Thickness measurements with paired point information |
| `*_thickness_stats.log` | Statistical summary (mean, median, distribution) |
| `*_vertices.mrc` | (Optional) Binary MRC with surface point positions |
| `*_vertices.xyz` | (Optional) Point cloud for external visualization |
| `*_thickness_volume.mrc` | (Optional) MRC with thickness values at surface positions |

## Analysis Examples

Below are examples of running the pipeline in different configurations. **The parallelization option (GPU nodes vs CPU threads) affects only the thickness measurement step.**

### Define input and output paths

In [3]:
# Inputs
segmentation_path = 'inputs/2738_z200to340_segmented.mrc'
config_path = 'inputs/config_2738.yml'
membrane_labels = {
    'NE': 1,
    'OMM': 2,
    'IMM': 3
}

# Output folder
output_dir = 'outputs/'

### Run the full pipeline with thickness measurements parallelized on GPU nodes or CPU threads

The membrane thickness analysis pipeline can be executed using either:
- A YAML configuration file specifying membrane labels (`config_file = 'path/to/config.yml'`)
- A Python dictionary mapping segmentation labels to membrane types

The pipeline automatically selects GPU acceleration when available for maximum performance, with a CPU fallback option for systems without compatible GPUs.

#### Outputs:
- CSV files containing:
  - Surface point coordinates and normal orientations
  - Comprehensive thickness measurements with paired point information
- Statistical summary log files with:
  - Mean, median, and range of thickness values
  - Distribution histograms
  - Coverage statistics

In [4]:
# GPU-parallelized with a specified config file

memthick.run_full_pipeline(segmentation_path,
    output_dir=output_dir,
    config_path=config_path, # Alternatively use membrane_labels=membrane_labels if config_path = None
    interpolate=True,
    interpolation_points=1, 
    refine_normals=True, 
    flip_normals=True,
    max_thickness=8.0,
    max_angle=1.0,
    direction='1to2',
    save_vertices_mrc=False, save_vertices_xyz=False, save_thickness_mrc=False, # Can be set to true
    batch_size=2000,
    use_gpu=True)

2025-03-11 13:25:09,589 - Starting full membrane thickness analysis pipeline for inputs/2738_z200to340_segmented.mrc
2025-03-11 13:25:10,041 - Step 1: Processing membrane segmentation
2025-03-11 13:25:10,083 - Using mesh sampling step size: 1
2025-03-11 13:25:10,084 - Reading segmentation from inputs/2738_z200to340_segmented.mrc...
2025-03-11 13:25:10,205 - Voxel size: 0.7840 nm
2025-03-11 13:25:10,206 - Origin: (np.float32(0.0), np.float32(0.0), np.float32(0.0))
2025-03-11 13:25:10,207 - 
Processing NE (label 1)
2025-03-11 13:25:10,530 - Extracting surface points with step size 1...
2025-03-11 13:25:15,867 - Extracted 371023 surface points
2025-03-11 13:25:15,942 - 
Interpolating surface points (before: 371023 vertices)
2025-03-11 13:25:31,679 - 
Point interpolation details:
2025-03-11 13:25:31,680 - Original vertices shape: (371023, 3)
2025-03-11 13:25:31,680 - Original normals shape: (371023, 3)
2025-03-11 13:25:31,681 - Shape of vertices post interpolation: (498069, 3)
2025-03-11 1

{'NE': {'input_csv': 'outputs/2738_z200to340_segmented_NE_vertices_normals.csv',
  'thickness_csv': 'outputs/2738_z200to340_segmented_NE_thickness.csv',
  'stats_file': 'outputs/2738_z200to340_segmented_NE_thickness_stats.log'},
 'OMM': {'input_csv': 'outputs/2738_z200to340_segmented_OMM_vertices_normals.csv',
  'thickness_csv': 'outputs/2738_z200to340_segmented_OMM_thickness.csv',
  'stats_file': 'outputs/2738_z200to340_segmented_OMM_thickness_stats.log'},
 'IMM': {'input_csv': 'outputs/2738_z200to340_segmented_IMM_vertices_normals.csv',
  'thickness_csv': 'outputs/2738_z200to340_segmented_IMM_thickness.csv',
  'stats_file': 'outputs/2738_z200to340_segmented_IMM_thickness_stats.log'}}

In [5]:
# CPU parallelized using a dictionary mapping segmentation labels to membrane instances

memthick.run_full_pipeline(segmentation_path,
    output_dir=output_dir,
    membrane_labels=membrane_labels, # instead of specifying a config file
    interpolate=True,
    interpolation_points=1, 
    refine_normals=True, 
    flip_normals=True,
    max_thickness=8.0,
    max_angle=1.0,
    direction='1to2',
    batch_size=2000,
    use_gpu=False,
    num_cpu_threads=8) # Adjust depending on your machine

2025-03-11 13:30:01,997 - Starting full membrane thickness analysis pipeline for inputs/2738_z200to340_segmented.mrc
2025-03-11 13:30:01,997 - Step 1: Processing membrane segmentation
2025-03-11 13:30:01,998 - Using mesh sampling step size: 1
2025-03-11 13:30:01,998 - Reading segmentation from inputs/2738_z200to340_segmented.mrc...
2025-03-11 13:30:01,999 - Voxel size: 0.7840 nm
2025-03-11 13:30:01,999 - Origin: (np.float32(0.0), np.float32(0.0), np.float32(0.0))
2025-03-11 13:30:02,000 - 
Processing NE (label 1)
2025-03-11 13:30:02,047 - Extracting surface points with step size 1...
2025-03-11 13:30:07,230 - Extracted 371023 surface points
2025-03-11 13:30:07,303 - 
Interpolating surface points (before: 371023 vertices)
2025-03-11 13:30:23,350 - 
Point interpolation details:
2025-03-11 13:30:23,351 - Original vertices shape: (371023, 3)
2025-03-11 13:30:23,351 - Original normals shape: (371023, 3)
2025-03-11 13:30:23,352 - Shape of vertices post interpolation: (498069, 3)
2025-03-11 1

{'NE': {'input_csv': 'outputs/2738_z200to340_segmented_NE_vertices_normals.csv',
  'thickness_csv': 'outputs/2738_z200to340_segmented_NE_thickness.csv',
  'stats_file': 'outputs/2738_z200to340_segmented_NE_thickness_stats.log'},
 'OMM': {'input_csv': 'outputs/2738_z200to340_segmented_OMM_vertices_normals.csv',
  'thickness_csv': 'outputs/2738_z200to340_segmented_OMM_thickness.csv',
  'stats_file': 'outputs/2738_z200to340_segmented_OMM_thickness_stats.log'},
 'IMM': {'input_csv': 'outputs/2738_z200to340_segmented_IMM_vertices_normals.csv',
  'thickness_csv': 'outputs/2738_z200to340_segmented_IMM_thickness.csv',
  'stats_file': 'outputs/2738_z200to340_segmented_IMM_thickness_stats.log'}}

### Run the surface point sampling step

This step processes the segmentation to extract surface points, compute normal vectors, and assign points to membrane leaflets.

#### Process:
- Surface extraction using marching cubes algorithm
- Point interpolation for improved surface coverage
- Normal vector calculation and refinement
- Surface assignment using principal component analysis (PCA)

#### Output:
- `*_vertices_normals.csv` file containing:
  - Surface point coordinates (in both voxel and physical units)
  - Normal vector orientations
  - Surface assignment flags (surface1/surface2)

> **Note:** Parallelization is not relevant for this step.

In [6]:
# Run only the surface extraction and assignment step
results = memthick.process_membrane_segmentation(
    segmentation_path=segmentation_path,
    output_dir=output_dir,
    membrane_labels=membrane_labels,
    interpolate=True,
    refine_normals=True,
    flip_normals=True
)

2025-03-11 13:51:53,643 - Using mesh sampling step size: 1
2025-03-11 13:51:53,644 - Reading segmentation from inputs/2738_z200to340_segmented.mrc...
2025-03-11 13:51:53,645 - Voxel size: 0.7840 nm
2025-03-11 13:51:53,645 - Origin: (np.float32(0.0), np.float32(0.0), np.float32(0.0))
2025-03-11 13:51:53,646 - 
Processing NE (label 1)
2025-03-11 13:51:53,689 - Extracting surface points with step size 1...
2025-03-11 13:51:58,864 - Extracted 371023 surface points
2025-03-11 13:51:58,939 - 
Interpolating surface points (before: 371023 vertices)
2025-03-11 13:52:14,866 - 
Point interpolation details:
2025-03-11 13:52:14,867 - Original vertices shape: (371023, 3)
2025-03-11 13:52:14,867 - Original normals shape: (371023, 3)
2025-03-11 13:52:14,868 - Shape of vertices post interpolation: (498069, 3)
2025-03-11 13:52:14,868 - Shape of normals post interpolation: (498069, 3)
2025-03-11 13:52:14,969 - After point interpolation: 498069 vertices
2025-03-11 13:52:14,970 - 
Refining normals with wei

### Run the thickness measurement step with pre-existing surface points

For cases where you've already generated surface points and want to measure thickness:

#### Requirements:
- Input CSV must contain columns:
  - `x_voxel`, `y_voxel`, `z_voxel` (coordinate positions)
  - `normal_x`, `normal_y`, `normal_z` (normal vector components)
  - `surface1`, `surface2` (boolean flags for surface assignment)

#### Options:
- Measurement direction: from surface 1 to 2 (`direction="1to2"`) or surface 2 to 1 (`direction="2to1"`)
- Parallelization option: GPU (`use_gpu=True`) or CPU with parallelization (`use_gpu=False, num_cpu_threads=8`)
- Measurement parameters: cone angle (`max_angle`) and maximum thickness (`max_thickness`)

This approach is useful for:
- Comparing measurements in both directions
- Applying different measurement parameters to the same surface points
- Processing large datasets in stages

In [8]:
# Define input csv path

points_csv = 'outputs/2738_z200to340_segmented_NE_vertices_normals.csv'  # CSV file with surface points and normals, file can be found in the link specified above


In [9]:
# Run only the thickness measurement step using a previously generated CSV file

thickness_csv, stats_file = memthick.measure_membrane_thickness(
    segmentation_path=segmentation_path,  # Still needed for voxel size
    input_csv=points_csv,
    output_dir=output_dir,
    max_thickness=8.0,
    max_angle=1.0,
    direction='1to2',  # Options: "1to2" or "2to1"
    use_gpu=True       # Set to False for CPU-only processing
)

2025-03-11 13:57:09,782 - Reading segmentation from inputs/2738_z200to340_segmented.mrc...
2025-03-11 13:57:09,784 - Voxel size: 0.7840 nm
2025-03-11 13:57:09,785 - Origin: (np.float32(0.0), np.float32(0.0), np.float32(0.0))
2025-03-11 13:57:09,786 - Loading data from outputs/2738_z200to340_segmented_NE_vertices_normals.csv...
2025-03-11 13:57:10,135 - CUDA GPU is available and will be used
2025-03-11 13:57:10,136 - Starting thickness measurement (1to2)...
2025-03-11 13:57:10,136 - Measuring thickness from surface 1 to surface 2...
2025-03-11 13:57:10,137 - Starting GPU thickness measurement with 498069 points...
2025-03-11 13:57:10,137 - Source points: 246748, Target points: 251321
2025-03-11 13:57:10,138 - Max thickness: 8.0 nm (10.20 voxels)
2025-03-11 13:57:10,138 - Max angle: 1.0 degrees
2025-03-11 13:57:10,150 - Launching CUDA kernel with 1946 blocks and 256 threads per block...
2025-03-11 13:57:10,921 - Retrieving results from GPU...
2025-03-11 13:57:10,928 - Processing matches 

In [ ]:
thickness_df = pd.read_csv(thickness_csv)
display(thickness_df[thickness_df['valid_measurement']]) # Display valid thickness measurements

,x_voxel,y_voxel,z_voxel,x_physical,y_physical,z_physical,normal_x,normal_y,normal_z,surface1,surface2,thickness,valid_measurement,paired_point_idx
9,1,46,998,0.784000,36.064002,782.432039,0.592076,0.721508,-0.358987,True,False,2.352000,True,28
10,2,46,998,1.568000,36.064002,782.432039,0.575491,0.723353,-0.381536,True,False,2.352000,True,29
11,1,46,999,0.784000,36.064002,783.216039,0.574963,0.667472,-0.473179,True,False,1.753077,True,31
12,2,46,999,1.568000,36.064002,783.216039,0.554461,0.667744,-0.496680,True,False,2.826752,True,13322
13,1,47,999,0.784000,36.848002,783.216039,0.576270,0.546025,-0.608087,True,False,1.108743,True,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498053,140,1009,284,109.760005,791.056039,222.656011,-0.084973,0.260844,0.961634,True,False,5.259232,True,485648
498054,140,1010,284,109.760005,791.840039,222.656011,-0.066612,0.256072,0.964360,True,False,5.020050,True,489095
498061,140,1019,282,109.760005,798.896040,221.088011,-0.116015,0.130506,0.984636,True,False,4.832901,True,485702
498062,139,1019,281,108.976005,798.896040,220.304011,-0.184262,0.151040,0.971202,True,False,6.173222,True,475277


### Output thickness measurements as motive lists

You can convert thickness measurements into motive lists (MOTL files) for visualization in tools like ChimeraX. The thickness values are stored as "cross-correlation" scores, allowing for color-coded visualization of membrane thickness.

#### Visualization in ChimeraX

The generated MOTL files can be loaded into ChimeraX for visualization:

1. Each point represents a location on the membrane surface
2. Points are oriented according to the surface normal directions
3. Color intensity corresponds to membrane thickness values
4. The subsampled version provides a clearer view of thickness patterns

This visualization approach allows for:
- Intuitive spatial mapping of thickness variations
- Correlation of thickness with membrane features
- Easy identification of regions with anomalous thickness
- Creation of publication-quality figures

#### Implementation Notes

- The code handles coordinate system conversion (MRC to MOTL)
- Normal vectors are converted to Euler angles using the ZXZ convention
- Optional subsampling creates less crowded visualizations
- Both membrane surfaces are represented as separate MOTL files

In [12]:
import pandas as pd
import numpy as np
from cryocat import cryomotl
from cryocat import geom

In [13]:
def create_thickness_motls(thickness_csv, subsample_fraction=None, random_seed=42):
    """
    Convert thickness measurements to motive lists for both membrane surfaces.
    
    Parameters
    ----------
    thickness_csv : str
        Path to CSV file with thickness measurements
    subsample_fraction : float, optional
        Fraction of points to sample (between 0 and 1)
    random_seed : int
        Random seed for reproducible subsampling
        
    Returns
    -------
    motl1, motl2 : cryomotl.Motl
        Motive lists for surface 1 and surface 2
    """
    # Load thickness data
    df = pd.read_csv(thickness_csv)
    valid_mask = df['valid_measurement']
    
    # Subsample if requested
    if subsample_fraction is not None:
        if not 0 < subsample_fraction <= 1:
            raise ValueError("subsample_fraction must be between 0 and 1")
            
        np.random.seed(random_seed)
        valid_indices = np.where(valid_mask)[0]
        n_samples = int(len(valid_indices) * subsample_fraction)
        sampled_indices = np.random.choice(valid_indices, size=n_samples, replace=False)
        
        subsample_mask = np.zeros_like(valid_mask)
        subsample_mask[sampled_indices] = True
        valid_mask = subsample_mask
    
    # Create surface 1 motive list
    motl1 = cryomotl.Motl()
    motl1_df = cryomotl.Motl.create_empty_motl_df()
    
    # Set coordinates (note axis permutation for coordinate system conversion)
    motl1_df['z'] = df.loc[valid_mask, 'x_voxel']
    motl1_df['y'] = df.loc[valid_mask, 'y_voxel']
    motl1_df['x'] = df.loc[valid_mask, 'z_voxel']
    motl1_df['score'] = df.loc[valid_mask, 'thickness']
    
    # Convert normal vectors to Euler angles
    normals = np.zeros((sum(valid_mask), 3))
    normals[:, 2] = df.loc[valid_mask, 'normal_x']
    normals[:, 1] = df.loc[valid_mask, 'normal_y']
    normals[:, 0] = df.loc[valid_mask, 'normal_z']
    
    euler_angles = geom.normals_to_euler_angles(normals, output_order="zxz")
    
    motl1_df['phi'] = euler_angles[:, 0]
    motl1_df['theta'] = euler_angles[:, 1]
    motl1_df['psi'] = euler_angles[:, 2]
    
    motl1.df = motl1_df
    
    # Create surface 2 motive list with paired points
    motl2 = cryomotl.Motl()
    motl2_df = cryomotl.Motl.create_empty_motl_df()
    
    paired_idx = df.loc[valid_mask, 'paired_point_idx'].values
    
    motl2_df['z'] = df.loc[paired_idx, 'x_voxel'].values
    motl2_df['y'] = df.loc[paired_idx, 'y_voxel'].values
    motl2_df['x'] = df.loc[paired_idx, 'z_voxel'].values
    motl2_df['score'] = df.loc[valid_mask, 'thickness'].values
    
    paired_normals = np.zeros((len(paired_idx), 3))
    paired_normals[:, 2] = df.loc[paired_idx, 'normal_x'].values
    paired_normals[:, 1] = df.loc[paired_idx, 'normal_y'].values
    paired_normals[:, 0] = df.loc[paired_idx, 'normal_z'].values
    
    paired_euler_angles = geom.normals_to_euler_angles(paired_normals, output_order="zxz")
    
    motl2_df['phi'] = paired_euler_angles[:, 0]
    motl2_df['theta'] = paired_euler_angles[:, 1]
    motl2_df['psi'] = paired_euler_angles[:, 2]
    
    motl2.df = motl2_df
    
    return motl1, motl2

def save_thickness_motls(thickness_csv, output_prefix, subsample_fraction=None):
    """
    Save thickness measurements as motive lists for visualization.
    
    Parameters
    ----------
    thickness_csv : str
        Path to CSV file with thickness measurements
    output_prefix : str
        Prefix for output MOTL files
    subsample_fraction : float, optional
        Fraction of points to sample (for creating reduced dataset)
    """
    # Create and save full resolution motls
    motl1, motl2 = create_thickness_motls(thickness_csv)
    motl1.write_out(f"{output_prefix}_surface1.em")
    motl2.write_out(f"{output_prefix}_surface2.em")
    
    # Create and save subsampled motls if requested
    if subsample_fraction is not None:
        sub_motl1, sub_motl2 = create_thickness_motls(
            thickness_csv, 
            subsample_fraction=subsample_fraction
        )
        sub_motl1.write_out(f"{output_prefix}_surface1_subsampled.em")
        sub_motl2.write_out(f"{output_prefix}_surface2_subsampled.em")
        
        print(f"Saved full motls with {len(motl1.df)} points")
        print(f"Saved subsampled motls with {len(sub_motl1.df)} points")
    else:
        print(f"Saved motls with {len(motl1.df)} points")


In [14]:
thickness_csv = 'outputs/2738_z200to340_segmented_OMM_thickness.csv'
output_prefix = 'outputs/2738_z200to340_segmented_OMM'
save_thickness_motls(thickness_csv, output_prefix=output_prefix, subsample_fraction=0.5)

Saved full motls with 76318 points
Saved subsampled motls with 38159 points
